In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config("Users/odl_user_281299/config.json")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

StatementMeta(a0ef7d74-67b7-4b47-be07-4fc81dcca125, 1, 6, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EFRC58YF5 to authenticate.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-281299
Azure region: eastus2
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-281299


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "project-1-cluster"

# DONE: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target: ", cluster_name)
except ComputeTargetException:
    print("Creating a new compute target: ", cluster_name)
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", min_nodes=0, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

StatementMeta(a0ef7d74-67b7-4b47-be07-4fc81dcca125, 1, 7, Finished, Available, Finished)

Creating a new compute target:  project-1-cluster
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
!pip install azureml-sdk
!pip install azureml.widgets

StatementMeta(a0ef7d74-67b7-4b47-be07-4fc81dcca125, 1, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 99.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 124.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.0/608.0 kB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 80.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 kB 40.5 MB/s eta 0:00:00
     ━━━━━

In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.01, 0.1, 1, 5),
    '--max_iter': choice(50, 100, 150, 250)
}) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=2,
) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='Users/odl_user_281299/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='Users/odl_user_281299/',
    script='train_filled.py',
    compute_target=compute_target,
    environment=sklearn_env
) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
) ### YOUR CODE HERE ###

StatementMeta(a0ef7d74-67b7-4b47-be07-4fc81dcca125, 1, 11, Finished, Available, Finished)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hd_run = exp.submit(config=hyperdrive_config)
RunDetails(hd_run).show()
hd_run.wait_for_completion(show_output=True)

StatementMeta(a0ef7d74-67b7-4b47-be07-4fc81dcca125, 1, 12, Finished, Available, Finished)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5
Web View: https://ml.azure.com/runs/HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-281299/workspaces/quick-starts-ws-281299&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-05-24T12:46:10.7076026Z][GENERATOR][WARNING]Space size : 16 is less than max total jobs : 20, only 16 jobs will be generated 
[2025-05-24T12:46:12.1977412Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-05-24T12:46:12.4708988Z][SCHEDULER][INFO]Scheduling job, id='HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5_0' 
[2025-05-24T12:46:12.4726233Z][SCHEDULER][INFO]Scheduling job, id='HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5_2' 
[2025-05-24T12:46:12.5204504Z][SCHEDULER][INFO]Scheduling job, id='HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5_3' 
[2025-05-24T12:46:12.4718774Z][SCHEDULER][INFO]Scheduling job, id='HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5_1' 
[2025-05-

{'runId': 'HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5',
 'target': 'project-1-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-05-24T12:46:10.430442Z',
 'endTimeUtc': '2025-05-24T12:54:17.028544Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '309835d6-0bac-4edd-96ec-3810b1947b34',
  'user_agent': 'python/3.10.6 (Linux-4.15.0-1002-azure-fips-x86_64-with-glibc2.27) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'best_child_run_id': 'HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5_14',
  'score': '0.9088012139605464',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5_14'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': '

In [11]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
model = best_run.register_model(model_name='best_model', model_path='outputs/model.joblib')
# joblib.dump(model, 'outputs/best_model.joblib')
print('Best run ID: ', best_run.id)
print("Parameters: ", best_run.get_details()['runDefinition']['arguments'])

StatementMeta(a0ef7d74-67b7-4b47-be07-4fc81dcca125, 1, 16, Finished, Available, Finished)

Best run ID:  HD_bb2b9240-8b6b-4a57-a64b-26f6fab52bd5_14
Parameters:  ['--C', '0.1', '--max_iter', '100']


In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

StatementMeta(, , -1, SessionStarting, , SessionStarting)

In [29]:
from Users.odl_user_281299.train_filled import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
ds = pd.read_csv('Users/odl_user_281299/bankmarketing_train.csv')
x, y = clean_data(ds)
training_data = pd.concat([x, y], axis=1)

StatementMeta(a0ef7d74-67b7-4b47-be07-4fc81dcca125, 1, 34, Finished, Available, Finished)

AttributeError: 'DataFrame' object has no attribute 'to_pandas_dataframe'

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_data,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_model = automl_run.get_output()
joblib.dumb(fitted_model, 'Users/odl_user_281299/outputs/best_automl_model.pkl')
print('Best model saved successfully!')

In [ ]:
compute_target.delete()